In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# create candidates
candidates = {0: ["Elizabeth II", 0.33], 1: ["Vladimir Putin", 0.22], 2: ["Xi Jinping", 0.17], 3: ["Muhammad bin Salman", 0.17], 4: ["Kim Jong Un", 0.11]}
candidates

def getProbs():
    return [candidates[0][1], candidates[1][1], candidates[2][1], candidates[3][1], candidates[4][1]]

In [8]:
def getVotePopulation(n):
    # create vote population
    votes = pd.DataFrame(index=range(n), columns=["Favorite", "Second Choice", "Eh", "Only Cause I Have To", "No Chance"]).fillna(-1)

    # FOR DEMONSTRATION PURPOSES
    # for i in range(5):
    #     newl = np.random.choice([0, 1, 2, 3, 4], size=5, replace=False, p=[0.96, 0.01, 0.01, 0.01, 0.01])
        # print(newl)
        # [0 4 2 1 3]
        # [0 2 4 1 3]
        # [0 2 4 3 1]
        # [0 3 2 4 1]
        # [0 3 1 4 2]

    vote_list = [0, 1, 2, 3, 4]
    ps = getProbs()
    for i in range(n):
        newl = np.random.choice(vote_list, size=len(vote_list), replace=False, p=ps)
        votes.at[i, "Favorite"] = newl[0]
        votes.at[i, "Second Choice"] = newl[1]
        votes.at[i, "Eh"] = newl[2]
        votes.at[i, "Only Cause I Have To"] = newl[3]
        votes.at[i, "No Chance"] = newl[4]
    return votes



,Favorite,Second Choice,Eh,Only Cause I Have To,No Chance
0,1,0,3,4,2


In [18]:
def getVoteCount(votes_df):
    # count votes
    vote_count = {}
    for i in range(5):
        indices = votes_df.loc[ votes_df["Favorite"].iloc[:] == i ].index.values
        vote_count[i] = len(indices)
    return vote_count



In [26]:
votes = getVotePopulation(10000000)
getVoteCount(votes)

{0: 5, 1: 1, 2: 1, 3: 2, 4: 1}